In [85]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Range1d, ColumnDataSource, HoverTool, CrosshairTool
from bokeh.layouts import gridplot
from bokeh.palettes import Category20

import pandas as pd
import numpy as np

import requests
import io

output_notebook()


Loading BokehJS ...

In [150]:
scenarios = ('historical', 'ssp1_rcp2.6_image', 'ssp3_rcp7.0_aim', 'ssp4_rcp3.4_gcam', 'ssp4_rcp6.0_gcam', 'ssp5_rcp8.5_remind-magpie')
plots = []
for scenario in scenarios:
    print(scenario)
    row = []
    for indicator in ('BIIAb', 'BIISR'):
        title = 'historical'
        base_url = "http://ipbes.s3.amazonaws.com/summary/%s-%s-%s- 900-2014.csv"
        if scenario != 'historical':
            ssp, rcp, model = scenario.upper().split('_')
            title = '%s -- %s / %s' % (indicator, ssp, rcp)
            base_url = "http://ipbes.s3.amazonaws.com/summary/%s-%s-%s-2015-2100.csv"
        p = figure(title=title)
        p.y_range = Range1d(0.45, 1)
        print(base_url % (scenario, indicator, 'subreg'))
        s = requests.get(base_url % (scenario, indicator, 'subreg')).content
        df = pd.read_csv(io.StringIO(s.decode('utf-8')))
        #df = pd.read_csv('ipbes-upload/%s-%s-subreg-2015-2100.csv' % (scenario, indicator))
        subset = df.loc[:, '2015':'2100'].T
        subset.columns = df['Name']
        subset.reset_index(inplace=True)
        subset = subset.rename(columns={'index': 'Year'})
        mypalette=Category20[len(subset.columns)]

        for idx, col in enumerate(subset.columns):
            if col in ('Year', 'Excluded'):
                continue
            src = ColumnDataSource(data={
                'year': subset.Year,
                'data': subset[col],
                'name': [col for n in range(len(subset))]
            })
            p.line('year', 'data', source=src, line_width=4, color=mypalette[idx])
            
        base_url = "http://ipbes.s3.amazonaws.com/summary/%s-%s-%s-2015-2100.csv"
        s = requests.get(base_url % (scenario, indicator, 'global')).content
        df = pd.read_csv(io.StringIO(s.decode('utf-8')))
        glob = df.loc[:, '2015':'2100'].T.reset_index()
        src = ColumnDataSource(data={
            'year': glob['index'],
            'data': glob[0],
            'name': ['Global' for n in range(len(glob))]
        })
        p.line('year', 'data', source=src, line_width=8, color='black')
        p.add_tools(HoverTool(tooltips=[('Year', '@year'), (indicator, '@data'), ('Region', '@name')]))
        row.append(p)
    plots.append(row)
grid = gridplot(plots, sizing_mode='scale_width')
show(grid)

historical
http://ipbes.s3.amazonaws.com/summary/historical-BIIAb-subreg- 900-2014.csv


KeyError: '2015'